In [1]:
import sys
import io
import time
import requests
import datetime
import numpy as np
import pandas as pd

path_root = '../..'
sys.path.append(f'{path_root}/')
from src.util.functions import Functions as F

# Disable warning
pd.options.mode.chained_assignment = None 

level = 'regioni'

path_data = f'{path_root}/data'
path_csv = f'{path_data}/csv'

csv_out_name = f'{path_csv}/covid19-{level}.csv'
pop_regions_name = f'{path_csv}/popolazione-regioni.csv'
icu_regions_name = f'{path_csv}/terapie-intensive.csv'
regions_name = f'{path_csv}/regioni.csv'

In [2]:
url_cum = F.get_url_cum(level)
url_cum

'https://raw.githubusercontent.com/pcm-dpc/COVID-19/master/dati-regioni/dpc-covid19-ita-regioni.csv'

In [3]:
df_raw_r = pd.read_csv(url_cum)
df_raw_r

,data,stato,codice_regione,denominazione_regione,lat,long,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,...,note,ingressi_terapia_intensiva,note_test,note_casi,totale_positivi_test_molecolare,totale_positivi_test_antigenico_rapido,tamponi_test_molecolare,tamponi_test_antigenico_rapido,codice_nuts_1,codice_nuts_2
0,2020-02-24T18:00:00,ITA,13,Abruzzo,42.351222,13.398438,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2020-02-24T18:00:00,ITA,17,Basilicata,40.639471,15.805148,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2020-02-24T18:00:00,ITA,18,Calabria,38.905976,16.594402,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2020-02-24T18:00:00,ITA,15,Campania,40.839566,14.250850,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2020-02-24T18:00:00,ITA,8,Emilia-Romagna,44.494367,11.341721,10,2,12,6,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7870,2021-03-04T17:00:00,ITA,19,Sicilia,38.115697,13.362357,676,118,794,23751,...,NaN,6.0,NaN,NaN,154701.0,0.0,1779124.0,761323.0,ITG,ITG1
7871,2021-03-04T17:00:00,ITA,9,Toscana,43.769231,11.255889,1029,173,1202,18501,...,NaN,12.0,NaN,NaN,159775.0,797.0,2518302.0,294210.0,ITI,ITI1
7872,2021-03-04T17:00:00,ITA,10,Umbria,43.106758,12.388247,436,82,518,7219,...,"Si fa presente che 12 dei ricoveri NON UTI, no...",5.0,NaN,NaN,45619.0,0.0,706761.0,128625.0,ITI,ITI2
7873,2021-03-04T17:00:00,ITA,2,Valle d'Aosta,45.737503,7.320149,13,2,15,153,...,NaN,0.0,NaN,NaN,8083.0,0.0,75117.0,3628.0,ITC,ITC2


In [4]:
list(df_raw_r.columns)

['data',
 'stato',
 'codice_regione',
 'denominazione_regione',
 'lat',
 'long',
 'ricoverati_con_sintomi',
 'terapia_intensiva',
 'totale_ospedalizzati',
 'isolamento_domiciliare',
 'totale_positivi',
 'variazione_totale_positivi',
 'nuovi_positivi',
 'dimessi_guariti',
 'deceduti',
 'casi_da_sospetto_diagnostico',
 'casi_da_screening',
 'totale_casi',
 'tamponi',
 'casi_testati',
 'note',
 'ingressi_terapia_intensiva',
 'note_test',
 'note_casi',
 'totale_positivi_test_molecolare',
 'totale_positivi_test_antigenico_rapido',
 'tamponi_test_molecolare',
 'tamponi_test_antigenico_rapido',
 'codice_nuts_1',
 'codice_nuts_2']

In [5]:
# Drop useless columns
cols_to_drop = ['stato', 'lat', 'long', 'casi_da_sospetto_diagnostico', 'casi_da_screening', 'note', 'note_test', 'note_casi']
df_rd = df_raw_r.drop(cols_to_drop, axis=1)
df_rd

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,...,totale_casi,tamponi,casi_testati,ingressi_terapia_intensiva,totale_positivi_test_molecolare,totale_positivi_test_antigenico_rapido,tamponi_test_molecolare,tamponi_test_antigenico_rapido,codice_nuts_1,codice_nuts_2
0,2020-02-24T18:00:00,13,Abruzzo,0,0,0,0,0,0,0,...,0,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2020-02-24T18:00:00,17,Basilicata,0,0,0,0,0,0,0,...,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2020-02-24T18:00:00,18,Calabria,0,0,0,0,0,0,0,...,0,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2020-02-24T18:00:00,15,Campania,0,0,0,0,0,0,0,...,0,10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2020-02-24T18:00:00,8,Emilia-Romagna,10,2,12,6,18,0,18,...,18,148,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7870,2021-03-04T17:00:00,19,Sicilia,676,118,794,23751,24545,-584,560,...,154701,2540447,1149114.0,6.0,154701.0,0.0,1779124.0,761323.0,ITG,ITG1
7871,2021-03-04T17:00:00,9,Toscana,1029,173,1202,18501,19703,444,1239,...,160572,2812512,1489353.0,12.0,159775.0,797.0,2518302.0,294210.0,ITI,ITI1
7872,2021-03-04T17:00:00,10,Umbria,436,82,518,7219,7737,-201,267,...,45619,835386,316639.0,5.0,45619.0,0.0,706761.0,128625.0,ITI,ITI2
7873,2021-03-04T17:00:00,2,Valle d'Aosta,13,2,15,153,168,2,11,...,8083,78745,44623.0,0.0,8083.0,0.0,75117.0,3628.0,ITC,ITC2


In [6]:
# Compute most recent couple of dates
last_date_str = df_rd.iloc[-1].data.split('T')[0]
last_date = datetime.datetime.strptime(last_date_str, '%Y-%m-%d')
prev_date = F.get_previous_date(last_date)
prev_date_str = datetime.datetime.strftime(prev_date, '%Y-%m-%d')
last_date_str, prev_date_str

('2021-03-04', '2021-03-03')

In [7]:
# Current state S
df_raw_r_1 = df_rd[df_rd.data.str.startswith(last_date_str)].sort_values(by='codice_regione').reset_index(drop=True)
df_raw_r_1

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,...,totale_casi,tamponi,casi_testati,ingressi_terapia_intensiva,totale_positivi_test_molecolare,totale_positivi_test_antigenico_rapido,tamponi_test_molecolare,tamponi_test_antigenico_rapido,codice_nuts_1,codice_nuts_2
0,2021-03-04T17:00:00,1,Piemonte,2171,188,2359,16798,19157,1226,2167,...,253602,2649872,1281529.0,19.0,247314.0,6288.0,2105421.0,544451.0,ITC,ITC1
1,2021-03-04T17:00:00,2,Valle d'Aosta,13,2,15,153,168,2,11,...,8083,78745,44623.0,0.0,8083.0,0.0,75117.0,3628.0,ITC,ITC2
2,2021-03-04T17:00:00,3,Lombardia,4735,532,5267,67127,72394,3058,5174,...,618742,6821426,3167660.0,56.0,607373.0,11369.0,6366989.0,454437.0,ITC,ITC4
3,2021-03-04T17:00:00,5,Veneto,866,131,997,25604,26601,837,1487,...,338237,5170985,1413578.0,14.0,333295.0,4942.0,4094228.0,1076757.0,ITH,ITH3
4,2021-03-04T17:00:00,6,Friuli Venezia Giulia,392,63,455,10485,10940,385,643,...,78799,1379290,491899.0,4.0,72885.0,5914.0,1272378.0,106912.0,ITH,ITH4
5,2021-03-04T17:00:00,7,Liguria,535,55,590,5095,5685,187,414,...,79406,1060675,450754.0,7.0,79406.0,0.0,944611.0,116064.0,ITC,ITC3
6,2021-03-04T17:00:00,8,Emilia-Romagna,2600,259,2859,45269,48128,1430,2545,...,270155,3983946,1529215.0,32.0,270009.0,146.0,3477649.0,506297.0,ITH,ITH5
7,2021-03-04T17:00:00,9,Toscana,1029,173,1202,18501,19703,444,1239,...,160572,2812512,1489353.0,12.0,159775.0,797.0,2518302.0,294210.0,ITI,ITI1
8,2021-03-04T17:00:00,10,Umbria,436,82,518,7219,7737,-201,267,...,45619,835386,316639.0,5.0,45619.0,0.0,706761.0,128625.0,ITI,ITI2
9,2021-03-04T17:00:00,11,Marche,580,87,667,9503,10170,258,919,...,70396,857713,522867.0,6.0,70396.0,0.0,786684.0,71029.0,ITI,ITI3


In [8]:
# State S - 1
df_raw_r_0 = df_rd[df_rd.data.str.startswith(prev_date_str)].sort_values(by='codice_regione').reset_index(drop=True)
df_raw_r_0

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,...,totale_casi,tamponi,casi_testati,ingressi_terapia_intensiva,totale_positivi_test_molecolare,totale_positivi_test_antigenico_rapido,tamponi_test_molecolare,tamponi_test_antigenico_rapido,codice_nuts_1,codice_nuts_2
0,2021-03-03T17:00:00,1,Piemonte,2140,181,2321,15610,17931,598,1537,...,251435,2631888,1276314.0,8.0,245418.0,6017.0,2098044.0,533844.0,ITC,ITC1
1,2021-03-03T17:00:00,2,Valle d'Aosta,12,2,14,152,166,3,9,...,8072,78370,44525.0,0.0,8072.0,0.0,74880.0,3490.0,ITC,ITC2
2,2021-03-03T17:00:00,3,Lombardia,4545,506,5051,64285,69336,1646,4590,...,613568,6767863,3151288.0,44.0,602818.0,10750.0,6328217.0,439646.0,ITC,ITC4
3,2021-03-03T17:00:00,5,Veneto,840,128,968,24796,25764,540,1272,...,336750,5129954,1409434.0,23.0,331808.0,4942.0,4078900.0,1051054.0,ITH,ITH3
4,2021-03-03T17:00:00,6,Friuli Venezia Giulia,386,62,448,10107,10555,349,693,...,78156,1370182,488423.0,7.0,72492.0,5664.0,1266399.0,103783.0,ITH,ITH4
5,2021-03-03T17:00:00,7,Liguria,526,53,579,4919,5498,71,368,...,78992,1053405,447907.0,3.0,78992.0,0.0,940261.0,113144.0,ITC,ITC3
6,2021-03-03T17:00:00,8,Emilia-Romagna,2528,251,2779,43919,46698,851,2456,...,267630,3945715,1520827.0,28.0,267487.0,143.0,3456816.0,488899.0,ITH,ITH5
7,2021-03-03T17:00:00,9,Toscana,1021,171,1192,18067,19259,431,1163,...,159333,2788768,1476328.0,14.0,158570.0,763.0,2503162.0,285606.0,ITI,ITI1
8,2021-03-03T17:00:00,10,Umbria,431,83,514,7424,7938,-161,301,...,45352,828523,315735.0,9.0,45352.0,0.0,703154.0,125369.0,ITI,ITI2
9,2021-03-03T17:00:00,11,Marche,561,84,645,9267,9912,140,759,...,69477,851158,518930.0,10.0,69477.0,0.0,781250.0,69908.0,ITI,ITI3


In [9]:
# New df regions state S with Trentino-Alto Adige
df_r_1 = df_raw_r_1
# df_r_1 = Functions.replace_bt_with_taa(df_raw_r_1)
# df_r_1.sort_values(by='codice_regione').reset_index(drop=True)

In [10]:
# New df regions state S - 1 with Trentino-Alto Adige
df_r_0 = df_raw_r_0
# df_r_0 = Functions.replace_bt_with_taa(df_raw_r_0)
# df_r_0.sort_values(by='codice_regione').reset_index()

In [11]:
df_r = df_r_1.copy()
# Merge with population data
df_pop_r = pd.read_csv(pop_regions_name)
df_m_r_1 = df_r.merge(df_pop_r, on='denominazione_regione')
df_m_r_1.sort_values(by='codice_regione').reset_index(drop=True)

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,...,tamponi,casi_testati,ingressi_terapia_intensiva,totale_positivi_test_molecolare,totale_positivi_test_antigenico_rapido,tamponi_test_molecolare,tamponi_test_antigenico_rapido,codice_nuts_1,codice_nuts_2,popolazione
0,2021-03-04T17:00:00,1,Piemonte,2171,188,2359,16798,19157,1226,2167,...,2649872,1281529.0,19.0,247314.0,6288.0,2105421.0,544451.0,ITC,ITC1,4.341375e+06
1,2021-03-04T17:00:00,2,Valle d'Aosta,13,2,15,153,168,2,11,...,78745,44623.0,0.0,8083.0,0.0,75117.0,3628.0,ITC,ITC2,1.255010e+05
2,2021-03-04T17:00:00,3,Lombardia,4735,532,5267,67127,72394,3058,5174,...,6821426,3167660.0,56.0,607373.0,11369.0,6366989.0,454437.0,ITC,ITC4,1.010397e+07
3,2021-03-04T17:00:00,5,Veneto,866,131,997,25604,26601,837,1487,...,5170985,1413578.0,14.0,333295.0,4942.0,4094228.0,1076757.0,ITH,ITH3,4.907704e+06
4,2021-03-04T17:00:00,6,Friuli Venezia Giulia,392,63,455,10485,10940,385,643,...,1379290,491899.0,4.0,72885.0,5914.0,1272378.0,106912.0,ITH,ITH4,1.211357e+06
5,2021-03-04T17:00:00,7,Liguria,535,55,590,5095,5685,187,414,...,1060675,450754.0,7.0,79406.0,0.0,944611.0,116064.0,ITC,ITC3,1.543127e+06
6,2021-03-04T17:00:00,8,Emilia-Romagna,2600,259,2859,45269,48128,1430,2545,...,3983946,1529215.0,32.0,270009.0,146.0,3477649.0,506297.0,ITH,ITH5,4.467118e+06
7,2021-03-04T17:00:00,9,Toscana,1029,173,1202,18501,19703,444,1239,...,2812512,1489353.0,12.0,159775.0,797.0,2518302.0,294210.0,ITI,ITI1,3.722729e+06
8,2021-03-04T17:00:00,10,Umbria,436,82,518,7219,7737,-201,267,...,835386,316639.0,5.0,45619.0,0.0,706761.0,128625.0,ITI,ITI2,8.802850e+05
9,2021-03-04T17:00:00,11,Marche,580,87,667,9503,10170,258,919,...,857713,522867.0,6.0,70396.0,0.0,786684.0,71029.0,ITI,ITI3,1.518400e+06


In [12]:
# Update icu csv data
df_icu_updated_r = pd.DataFrame.from_dict(F.icu_data())
df_icu_updated_r.to_csv(icu_regions_name, index=False)

In [13]:
# Merge with icu data
df_icu_r = pd.read_csv(icu_regions_name)
df_m_r_2 = df_m_r_1.merge(df_icu_r, left_on='denominazione_regione', right_on='regione')
df_m_r_2.sort_values(by='codice_regione').reset_index(drop=True)

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,...,tamponi_test_antigenico_rapido,codice_nuts_1,codice_nuts_2,popolazione,regione,ricoverati_area_non_critica,posti_letto_area_non_critica,ricoverati_terapia_intensiva,posti_letto_terapia_intensiva,posti_letto_terapia_intensiva_attivabili
0,2021-03-04T17:00:00,1,Piemonte,2171,188,2359,16798,19157,1226,2167,...,544451.0,ITC,ITC1,4.341375e+06,Piemonte,2140,5824,181,628,99
1,2021-03-04T17:00:00,2,Valle d'Aosta,13,2,15,153,168,2,11,...,3628.0,ITC,ITC2,1.255010e+05,Valle d'Aosta,12,227,2,20,12
2,2021-03-04T17:00:00,3,Lombardia,4735,532,5267,67127,72394,3058,5174,...,454437.0,ITC,ITC4,1.010397e+07,Lombardia,4545,10936,506,1416,114
3,2021-03-04T17:00:00,5,Veneto,866,131,997,25604,26601,837,1487,...,1076757.0,ITH,ITH3,4.907704e+06,Veneto,840,6000,128,1000,0
4,2021-03-04T17:00:00,6,Friuli Venezia Giulia,392,63,455,10485,10940,385,643,...,106912.0,ITH,ITH4,1.211357e+06,Friuli Venezia Giulia,386,1277,62,175,0
5,2021-03-04T17:00:00,7,Liguria,535,55,590,5095,5685,187,414,...,116064.0,ITC,ITC3,1.543127e+06,Liguria,526,1795,53,216,18
6,2021-03-04T17:00:00,8,Emilia-Romagna,2600,259,2859,45269,48128,1430,2545,...,506297.0,ITH,ITH5,4.467118e+06,Emilia-Romagna,2528,6126,251,757,0
7,2021-03-04T17:00:00,9,Toscana,1029,173,1202,18501,19703,444,1239,...,294210.0,ITI,ITI1,3.722729e+06,Toscana,1021,5033,171,577,19
8,2021-03-04T17:00:00,10,Umbria,436,82,518,7219,7737,-201,267,...,128625.0,ITI,ITI2,8.802850e+05,Umbria,431,852,83,141,2
9,2021-03-04T17:00:00,11,Marche,580,87,667,9503,10170,258,919,...,71029.0,ITI,ITI3,1.518400e+06,Marche,561,1176,84,233,40


In [14]:
# Copy last merged df into df_r
df_r = df_m_r_2.copy()

In [15]:
# Add columns
df_r['variazione_ricoverati_con_sintomi'] = df_r_1['ricoverati_con_sintomi'] - df_r_0['ricoverati_con_sintomi']
df_r['variazione_terapia_intensiva'] = df_r_1['terapia_intensiva'] - df_r_0['terapia_intensiva']
df_r['variazione_totale_ospedalizzati'] = df_r_1['totale_ospedalizzati'] - df_r_0['totale_ospedalizzati']
df_r['posti_disponibili_terapia_intensiva'] = df_r['posti_letto_terapia_intensiva'] - df_r['terapia_intensiva']
df_r['saturazione_terapia_intensiva'] = ((df_r_1['terapia_intensiva'] / df_r['posti_letto_terapia_intensiva']) * 100).round(2)
df_r['variazione_isolamento_domiciliare'] = df_r_1['isolamento_domiciliare'] - df_r_0['isolamento_domiciliare']
df_r['variazione_nuovi_positivi'] = df_r_1['nuovi_positivi'] - df_r_0['nuovi_positivi']
df_r['nuovi_tamponi'] = df_r_1['tamponi'] - df_r_0['tamponi']
df_r['nuovi_casi_testati'] = df_r_1['casi_testati'] - df_r_0['casi_testati']
df_r['nuovi_deceduti'] = df_r_1['deceduti'] - df_r_0['deceduti']
df_r['nuovi_dimessi_guariti'] = df_r_1['dimessi_guariti'] - df_r_0['dimessi_guariti']
df_r['np_su_nt'] = ((df_r['nuovi_positivi'] / df_r['nuovi_tamponi']) * 100).round(2)
df_r['np_su_p'] = ((df_r['nuovi_positivi'] / df_r['popolazione']) * 100).round(5)
df_r['tc_su_p'] = ((df_r['totale_casi'] / df_r['popolazione']) * 100).round(2)

df_r = df_r.sort_values(by='nuovi_positivi', ascending=False).reset_index(drop=True)
df_r

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,...,saturazione_terapia_intensiva,variazione_isolamento_domiciliare,variazione_nuovi_positivi,nuovi_tamponi,nuovi_casi_testati,nuovi_deceduti,nuovi_dimessi_guariti,np_su_nt,np_su_p,tc_su_p
0,2021-03-04T17:00:00,3,Lombardia,4735,532,5267,67127,72394,3058,5174,...,37.57,2842,584,53563,16372.0,59,2057,9.66,0.05121,6.12
1,2021-03-04T17:00:00,15,Campania,1357,140,1497,81461,82958,1706,2780,...,22.58,1702,145,23988,16329.0,40,1034,11.59,0.04805,4.79
2,2021-03-04T17:00:00,8,Emilia-Romagna,2600,259,2859,45269,48128,1430,2545,...,34.21,1350,89,38231,8388.0,25,1070,6.66,0.05697,6.05
3,2021-03-04T17:00:00,1,Piemonte,2171,188,2359,16798,19157,1226,2167,...,29.94,1188,630,17984,5215.0,26,915,12.05,0.04992,5.84
4,2021-03-04T17:00:00,12,Lazio,1860,241,2101,33942,36043,254,1702,...,25.56,218,182,33725,19559.0,22,1426,5.05,0.02902,4.09
5,2021-03-04T17:00:00,5,Veneto,866,131,997,25604,26601,837,1487,...,13.10,808,215,41031,4144.0,20,630,3.62,0.03030,6.89
6,2021-03-04T17:00:00,16,Puglia,1287,162,1449,32357,33806,138,1438,...,28.47,137,177,10058,4610.0,32,1268,14.30,0.03588,3.78
7,2021-03-04T17:00:00,9,Toscana,1029,173,1202,18501,19703,444,1239,...,29.98,434,76,23744,13025.0,23,772,5.22,0.03328,4.31
8,2021-03-04T17:00:00,11,Marche,580,87,667,9503,10170,258,919,...,37.34,236,160,6555,3937.0,9,652,14.02,0.06052,4.64
9,2021-03-04T17:00:00,6,Friuli Venezia Giulia,392,63,455,10485,10940,385,643,...,36.00,378,-50,9108,3476.0,10,248,7.06,0.05308,6.51


In [16]:
# Write pre-processed data
df_r.to_csv(csv_out_name, index=False)

In [17]:
# ICU Stats
df_r[['denominazione_regione', 'terapia_intensiva', 'posti_disponibili_terapia_intensiva', 'posti_letto_terapia_intensiva', 'saturazione_terapia_intensiva']] \
    .sort_values(by='saturazione_terapia_intensiva', ascending=False).reset_index(drop=True)

,denominazione_regione,terapia_intensiva,posti_disponibili_terapia_intensiva,posti_letto_terapia_intensiva,saturazione_terapia_intensiva
0,Umbria,82,59,141,58.16
1,Molise,21,18,39,53.85
2,P.A. Trento,46,44,90,51.11
3,Abruzzo,85,121,206,41.26
4,Lombardia,532,884,1416,37.57
5,Marche,87,146,233,37.34
6,Friuli Venezia Giulia,63,112,175,36.00
7,P.A. Bolzano,36,64,100,36.00
8,Emilia-Romagna,259,498,757,34.21
9,Toscana,173,404,577,29.98


In [18]:
def add_cols(df):
    df['variazione_terapia_intensiva'] = df['terapia_intensiva'].diff()
    df['variazione_ricoverati_con_sintomi'] = df['ricoverati_con_sintomi'].diff()
    df['variazione_totale_ospedalizzati'] = df['totale_ospedalizzati'].diff()
    df['nuovi_dimessi_guariti'] = df['dimessi_guariti'].diff()
    df['nuovi_deceduti'] = df['deceduti'].diff()
    df['nuovi_tamponi'] = df['tamponi'].diff()
    df['nuovi_casi_testati'] = df['casi_testati'].diff()
    df['variazione_isolamento_domiciliare'] = df['isolamento_domiciliare'].diff()
    df = df.iloc[1:]
    df['ti_su_to'] = ((df['terapia_intensiva'] / df['totale_ospedalizzati']).replace(np.inf, 0) * 100).round(2)
    df['to_su_tp'] = ((df['totale_ospedalizzati'] / df['totale_positivi']).replace(np.inf, 0) * 100).round(2)
    df['np_su_nt'] = ((df['nuovi_positivi'] / df['nuovi_tamponi']).replace(np.inf, 0) * 100).round(2)
    df['np_su_nct'] = ((df['nuovi_positivi'] / df['nuovi_casi_testati']).replace(np.inf, 0) * 100).round(2)
    df['var_ti_perc'] = ((df['variazione_terapia_intensiva'] / (df['terapia_intensiva'] - df['variazione_terapia_intensiva'])).replace(np.inf, 0) * 100).round(2)
    df['var_pos_perc'] = ((df['variazione_totale_positivi'] / (df['totale_positivi'] - df['variazione_totale_positivi'])).replace(np.inf, 0) * 100).round(2)
    return df.fillna(0)

In [19]:
# df_rd.sort_values(by='codice_regione').reset_index(drop=True)
df_rn = pd.read_csv(regions_name)
for region_code in df_rn.codice_regione:
    if region_code != 4: # TAA
        df_cur_region = df_rd[df_rd.codice_regione == region_code]
        region_name = df_cur_region.denominazione_regione.iloc[0].lower()
        csv_out_region_name = '{path_csv}/regioni/covid19-{region_name}.csv'.format(path_csv=path_csv, region_name=region_name)
        df_cur_region = add_cols(df_cur_region)
        df_cur_region.to_csv(csv_out_region_name, index=False)
        print('Aggiornato {file}'.format(file=csv_out_region_name))

Aggiornato ../../data/csv/regioni/covid19-piemonte.csv
Aggiornato ../../data/csv/regioni/covid19-valle d'aosta.csv
Aggiornato ../../data/csv/regioni/covid19-lombardia.csv
Aggiornato ../../data/csv/regioni/covid19-veneto.csv
Aggiornato ../../data/csv/regioni/covid19-friuli venezia giulia.csv
Aggiornato ../../data/csv/regioni/covid19-liguria.csv
Aggiornato ../../data/csv/regioni/covid19-emilia-romagna.csv
Aggiornato ../../data/csv/regioni/covid19-toscana.csv
Aggiornato ../../data/csv/regioni/covid19-umbria.csv
Aggiornato ../../data/csv/regioni/covid19-marche.csv
Aggiornato ../../data/csv/regioni/covid19-lazio.csv
Aggiornato ../../data/csv/regioni/covid19-abruzzo.csv
Aggiornato ../../data/csv/regioni/covid19-molise.csv
Aggiornato ../../data/csv/regioni/covid19-campania.csv
Aggiornato ../../data/csv/regioni/covid19-puglia.csv
Aggiornato ../../data/csv/regioni/covid19-basilicata.csv
Aggiornato ../../data/csv/regioni/covid19-calabria.csv
Aggiornato ../../data/csv/regioni/covid19-sicilia.csv


In [20]:
# df_tb = df_rd[df_rd.codice_regione > 20].reset_index(drop=True)
# df_tb

In [21]:
# Initialize df_taa
# df_taa = Functions.replace_bt_with_taa(df_tb[:2])
# for i in range(2, df_tb.shape[0], 2):
#     df_taa = df_taa.append(Functions.replace_bt_with_taa(df_tb[i:i+2]), ignore_index=True)
# df_taa = add_cols(df_taa)
# df_taa

In [22]:
# region_name = df_taa.denominazione_regione.iloc[0].lower()
# csv_out_region_name = '{path_csv}/regioni/covid19-{region_name}.csv'.format(path_csv=path_csv, region_name=region_name)
# df_taa.to_csv(csv_out_region_name, index=False)
# print('Aggiornato {file}'.format(file=csv_out_region_name))